In [1]:
from collections import Counter
import re
import numpy as np
from sklearn import datasets, linear_model
from scipy import stats as ss

file = ["google.com",
    "youtube.com",
    "facebook.com",
    "baidu.com",
    "yahoo.com",
    "wikipedia.org",
    "amazon.com",
    "qq.com",
    "twitter.com",
    "google.co.in",
    "live.com",
    "taobao.com",
    "bing.com",
    "google.co.jp",
    "msn.com",
    "yahoo.co.jp",
    "sina.com.cn",
    "linkedin.com",
    "weibo.com",
    "vk.com",
    "instagram.com",
    "google.ru",
    "yandex.ru",
    "hao123.com",
    "google.de",
    "ebay.com",
    "reddit.com",
    "google.co.uk",
    "google.com.br",
    "amazon.co.jp",
    "t.co",
    "google.fr",
    "pinterest.com",
    "mail.ru",
    "tmall.com",
    "360.cn",
    "microsoft.com",
    "netflix.com",
    "onclickads.net",
    "google.it",
    "sohu.com",
    "wordpress.com",
    "tumblr.com",
    "google.es",
    "blogspot.com",
    "imgur.com",
    "naver.com",
    "paypal.com",
    "stackoverflow.com",
    "chinadaily.com.cn",
    "xvideos.com",
    "apple.com",
    "google.com.mx",
    "aliexpress.com",
    "gmw.cn",
    "imdb.com",
    "fc2.com",
    "google.co.kr",
    "diply.com",
    "github.com",
    "google.ca",
    "pornhub.com",
    "google.com.hk",
    "amazon.de",
    "office.com",
    "ok.ru",
    "whatsapp.com",
    "google.com.tr",
    "alibaba.com",
    "rakuten.co.jp",
    "jd.com",
    "ask.com",
    "google.co.id",
    "kat.cr",
    "nicovideo.jp",
    "craigslist.org",
    "google.pl",
    "pixnet.net",
    "xinhuanet.com",
    "tianya.cn",
    "soso.com",
    "xhamster.com",
    "blogger.com",
    "googleusercontent.com",
    "amazon.in",
    "go.com",
    "dropbox.com",
    "google.com.au",
    "cntv.cn",
    "amazon.co.uk",
    "bongacams.com",
    "outbrain.com",
    "youku.com",
    "coccoc.com",
    "wikia.com",
    "microsoftonline.com",
    "youth.cn",
    "cnn.com",
    "google.com.tw",
    "163.com"]

category = {}
category["www.google.com"] = 0
category["www.facebook.com"] = 1
category["www.youtube.com"] = 2
category["www.baidu.com"] = 0
category["www.yahoo.com"] = 1
category["www.amazon.com"] = 2
category["www.wikipedia.org"] = 3
category["www.twitter.com"] = 1
category["www.google.co.in"] = 0
category["www.linkedin.com"] = 1
category["www.yahoo.co.jp"] = 1
category["www.google.co.jp"] = 0
category["www.google.ca"] = 0
category["www.qq.com"] = 2
category["www.taobao.com"] = 2
category["www.live.com"] = 1
category["www.google.com.mx"] = 0
category["www.google.com.hk"] = 0
category["www.pornhub.com"] = 2
category["www.alibaba.com"] = 2
category["www.amazon.co.uk"] = 2
category["www.google.com.tr"] = 0
category["www.adobe.com"] = 0
category["www.gmail.com"] = 0
category["www.xhamster.com"] = 2
category["www.ebay.de"] = 2
category["www.google.pl"] = 0
category["www.google.com.au"] = 0
category["www.dailymotion.com"] = 2
category["www.bbc.co.uk"] = 2
category["www.google.co.id"] = 0
category["www.flipkart.com"] = 2
category["www.cnn.com"] = 2
category["www.booking.com"] = 2
category["www.github.com"] = 0
category["www.dropbox.com"] = 0
category["www.people.com.cn"] = 2
category["www.nicovideo.jp"] = 2
category["www.googleusercontent.com"] = 0
category["www.weibo.com"] = 2
category["www.sina.com.cn"] = 2
category["www.adcash.com"] = 0
category["www.ok.ru"] = 2
category["www.diply.com"] = 2
category["www.tianya.cn"] = 2
category["www.jd.com"] = 2
category["www.cntv.cn"] = 0
category["www.go.com"] = 2
category["www.naver.com"] = 0
category["www.kat.cr"] = 0
category["www.soso.com"] = 0
category["www.wikia.com"] = 0
category["www.163.com"] = 2
category["www.gmw.cn"] = 2
category["www.pixnet.net"] = 3
category["www.outbrain.com"] = 3
category["www.xinhuanet.com"] = 2
category["www.rakuten.co.jp"] = 2
category["www.detail.tmall.com"] = 0
category["www.chinadaily.com.cn"] = 0

def find_PLT(filename):
    file = open(filename)
    list = file.readlines()
    mintime = 0
    for line in list:
        match = re.match('.*requestTime.: ([0-9^.]+)', line)
        if match is not None:
            mintime = float(match.groups(0)[0])
            break
    maxtime = 0
    for line in list:
        match = re.match('.*time.: ([0-9^.]+)',line)
        if match is not None:
            if float(match.groups(0)[0]) > maxtime:
                maxtime = float(match.groups(0)[0])
                
    length = 0
    for line in list:
        match = re.match('.*"len": ([0-9]+), "from',line)
        if match is not None:
            for val in match.groups(0):
                length += int(val)                
    
    return (maxtime - mintime,mintime,maxtime,length)
            
def findcount(text, list):
    feature = Counter()
    for i in range(4):
        feature[list[i]] = len(re.findall('\.' + list[i],text))
#         print list[i],":",feature[list[i]]
    for i in range(4,len(list)):
        feature[list[i]] = len(re.findall(list[i],text))
    return feature


def findAllPLT(name,count):
    list = []
    featureList = ['png','gif','css','js','Resource']
    feature = Counter()
    length = 0
    for i in range(count):
        filename = name + str(i)  + ".txt"
        text = open(filename).read()
        feature += findcount(text,featureList)
        (time,min,max,length) = find_PLT(filename)
        list.append(time)
    
    for key in featureList:
        if key in feature:
            feature[key] = (feature[key]*1.0)/count
        else:
            feature[key] = 0
    feature['length'] = length
    return feature,list

In [6]:
filepath = "/Users/Aman/Desktop/Colin/data_all/new_logs/"
def all_file():
    dictionary = {}
    for website in file:
        feature,list = findAllPLT(filepath + website,10)
        dictionary[website] = (feature,list)
    return dictionary

all_val_dict = all_file()

In [9]:
def regression(Y,X):
    X = np.array(X)
    Y = np.array(Y)
    Y = np.array(ss.zscore(Y)).T
#     X = np.array([x + 0.1001 for x in X])
    X = np.array([(ss.zscore(x)) for x in X.T]).T
    
    regr = linear_model.LinearRegression()
    regr.fit(X,Y)
    print('Coefficients: \n', regr.coef_)
    print("Residual sum of squares: %.2f"% np.mean((regr.predict(X) - Y) ** 2))
    
    return regr    

def getValueRegr(all_val_dict):
    Y = []
    X = []
    for website in file:
        if np.sum(all_val_dict[website][1]) == 0:
            continue
        Y.append(np.mean(all_val_dict[website][1][4]))
        x1 = all_val_dict[website][0].values()
        
        X.append(x1)
    
    return (Y,X)

Y,X = getValueRegr(all_val_dict)

regr = regression(Y,X)
test = all_val_dict['www.yahoo.com'][0].values()
print test
# test = test[1:-1]
test = np.array(ss.zscore(test)).T
print test
print regr.predict(test)


for website in file:
    test = all_val_dict[website][0].values()
    test = np.array(ss.zscore(test)).T
    print regr.predict(test)
    print all_val_dict[website][1][0]

('Coefficients: \n', array([ 0.30561588,  0.13275116, -0.04940217,  0.31840384, -0.01278579,
       -0.183342  ]))
Residual sum of squares: 0.66


KeyError: 'www.yahoo.com'

In [10]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
def pageloaditeration():    
    for website in file:
        if website in all_val_dict.keys():
            if max(all_val_dict[website][1]) < 8 and min(all_val_dict[website][1])  > 0:
#                 all_val_dict[website][1].sort()
#                 all_val_dict[website][1].reverse()
                plt.plot(all_val_dict[website][1],linewidth=2.0)

    plt.ylabel('Time in seconds ---->')
    plt.xlabel('Iterations ---->')
    plt.ylim(ymax=8)
    plt.xlim(xmax=7)
    plt.grid(True)
    plt.title("PAGELOAD TIME  vs  ITERATION")
    plt.show()

def scatterplot(file):
    for website in file:
       plt.scatter(sum(all_val_dict[website][0].values()),np.mean(all_val_dict[website][1]),alpha=0.5,color="blue")

    plt.ylabel('Time in seconds ---->')
    plt.xlabel('Number of objects ---->')
    plt.grid(True)
    plt.title("PAGELOAD TIME  vs  #OBJECTS")
    plt.show()

def pageloadvsobj(all_val_dict):
    for website in file:
        sum = 0.0
        i = 0
        for key in all_val_dict[website][0]:
            sum += regr.coef_[i]* all_val_dict[website][0][key]
            i += 1
            sum = max(sum,2)
            plt.scatter(sum,np.mean(all_val_dict[website][1]),alpha=0.5)
    
    plt.ylabel('Time in seconds ---->')
    plt.xlabel('Number of objects ---->')
    plt.grid(True)
    plt.xlim(xmin=-5)
    plt.ylim(ymin=0)
    plt.title("PAGELOAD TIME  vs  #OBJECTS")
    plt.show()

def sp():
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    x = []
    y = []
    z = []
    yaxis = []
    keys = ['png','gif','css','js']
    for website in file:
        sum = 0
        for v in keys:
            sum += all_val_dict[website][0][v]
        x.append(sum)
        y.append(all_val_dict[website][0]["length"])
        z.append(all_val_dict[website][0]["Resource"])
        yaxis.append(np.mean(all_val_dict[website][1]))
    
    x = np.array(ss.zscore(x))
    y = np.array(ss.zscore(y))
    z = np.array(ss.zscore(z))
   
    colors = iter(cm.rainbow(np.linspace(0, 1, 3)))

    ax1.scatter(x,yaxis , s=35,color=next(colors), marker="o",edgecolor = 'black', label='#Objects')
    ax1.scatter(y,yaxis , s=35,color=next(colors), marker="s",edgecolor = 'black', label='LengthWebsite')
    ax1.scatter(z,yaxis , s=35,color=next(colors), marker="p",edgecolor = 'black', label='#Resources Contacted')
    plt.legend(loc='upper left');
    plt.colors()
    plt.ylabel('Time in seconds ---->')
    plt.xlabel('Features(NORMALIZED TO FIT/COMPARE) ---->')
    plt.grid(True)
    plt.title("PAGELOAD TIME vs Features")
    plt.show()

pageloaditeration()
pageloadvsobj(all_val_dict)
scatterplot(file)
sp()

In [ ]:
import pandas as pd

filepath = '/Users/Aman/Desktop/Colin/'
df = pd.read_csv(filepath + 'website_metrics.txt')

In [ ]:
Y = df['median_page_download_time']
X = df[['number_of_servers_contacted',
        'number_of_object_requests_median',
        'object_request_size_median',
        'number_of_javascript_objects_median',
       'number_of_javascript_objects_median',
       'size_of_javascript_objects_median',
       'number_of_image_objects_median',
       'size_of_image_objects_median',
       'number_of_flash_objects_median',
       'size_of_flash_objects_median',
       'number_of_css_objects_median',
       'size_of_css_objects_median',
       ]]